##Install necessary libraries

In [ ]:
!pip install -q transformers datasets
!pip install -q peft
!pip install -q accelerate
!pip install -U bitsandbytes
!pip install -q sacrebleu
!pip install -q rouge_score
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", 

## Load and clean MIMIC-IV-Notes dataset

In [ ]:
from pprint import pprint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def clean_text(text):
  text = text.replace('\n', ' ')
  text = ' '.join(text.split())
  return text

sample_size = 10000

In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/discharge.csv', keep_default_na = False)

data = data.sample(n=sample_size, random_state=42)
data['text'] = data['text'].apply(clean_text)

In [ ]:
import re

def extract_sections(text):
    # Regex to match the discharge instructions section
    discharge_pattern = re.compile(r"Discharge Instructions:(.*)", re.DOTALL)
    match = discharge_pattern.search(text)
    if match:
        discharge_summary = match.group(1).strip()
        rest_of_text = text.replace(discharge_summary, "").strip()
        return rest_of_text, discharge_summary
    return None, None

input_texts = []
output_texts = []

for index, row in data.iterrows():
    input_text, discharge_instructions = extract_sections(row['text'])
    if input_text and discharge_instructions:
        input_texts.append(input_text)
        output_texts.append(discharge_instructions)

In [ ]:
input_texts[0]

"Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: F Service: MEDICINE Allergies: Anticholinergics,Other / Reglan Attending: ___. Chief Complaint: Abdominal pain Major Surgical or Invasive Procedure: None History of Present Illness: Patient is a ___ yo woman with history of chronic pancreatitis s/p cholecystectomy and sphincterotomy who presents with 1wk of worsening abdominal pain. As per patient, the pain is intermittent, sharp and ___ in quality. It localizes to her mid/righ upper abdomen and radiates up the chest wall and to the back. Patient finds this pain to be very similar to her prior pancreatitis flare-ups. Denies precipitants including alcohol use, abd. trauma, infections or h/o gallstones. She notes that she is on a restricted diet (no caffeine/fatty food/fried food/dairy) as part of her pancreatitis management. Also notes that pain worsens with meals; and that it is alleviated when NPO, or with NSAIDs and dilaudid. . Patient was initall

In [ ]:
output_texts[0]

'You were admitted to the hospital because of abdominal pain and inibility to eat secondary to pain. You were given IV pain medication and fluids. Over the course of your stay you were slowly able to eat more starting with fluids first. You tolerated a low residue diet with your baseline amount of pain. You were seen by the ___ doctors in the hospital who recommended tests to be sent out to look for a cause of your pain. No cause could be found. Medication changes: You were started on Omeprazole 20mg once a day. Remember to avoid dairy products and try to eat a low residue bland diet. Your appointment with Dr. ___ was changed to ___ at 11am. Please return to the hospital or call your doctor if you have temperature greater than 101, shortness of breath, worsening difficulty with swallowing, chest pain, abdominal pain, diarrhea, or any other symptoms that you are concerned about. Followup Instructions: ___'

In [ ]:
len(output_texts)

9925

In [ ]:
from datasets import Dataset
dataset = []

for input_text, output_text in zip(input_texts, output_texts):
    dataset.append({
        "input": input_text,
        "output": output_text
    })

dataset = Dataset.from_list(dataset)
train_test_dataset = dataset.train_test_split(test_size=0.2)
test_valid = train_test_dataset['test'].train_test_split(test_size=0.5)
train = train_test_dataset["train"]
valid = test_valid["train"]
test = test_valid["test"]

In [ ]:
train

Dataset({
    features: ['input', 'output'],
    num_rows: 7940
})

In [ ]:
from transformers import BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
train[0]['input']

"Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: M Service: MEDICINE Allergies: naproxen / ibuprofen Attending: ___. Chief Complaint: Back pain Major Surgical or Invasive Procedure: None this hospitalization History of Present Illness: HISTORY OF PRESENT ILLNESS: This patient is a ___ year old male who complains of low back pain w/ herniated disc (L 3-L4) and previous laminectomy L2-L5 last year @ ___- presents w/ ___ pain- difficulty walking. Patient states that he has been having left-sided back pain radiating down to his left hip. No inc stool/urine. +chronic constipation (not assoc with onset of back pain). Took 25mg Oxycodone last night and fell OOB after this- No head trauma- Usually takes 20mg Oxycodone for pain exacerbations. No fevers or chills. No chest pain or shortness of breath. In the ED, initial vitals were: Temp: 98.8 HR: 101 BP: 132/74 Resp: 16 O(2)Sat: 95 Normal On the floor, Patient had notes regarding the h/o his back pain. He 

##Baseline

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model_name = 'google/pegasus-xsum'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(model_name)

config

PegasusConfig {
  "_name_or_path": "google/pegasus-xsum",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,


In [ ]:
import sacrebleu
# Function to generate discharge summary
def generate_discharge_summary(input_text):
    if not input_text:
        return ""

    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding='longest')

    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=200, num_beams=4, length_penalty=2.0, early_stopping=True)

    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(summary)
    return summary

# Convert Hugging Face Dataset to list of dictionaries
train_list = train.to_dict()

generated_summaries = []
reference_summaries = []

num_examples = 10

for i in range(num_examples):
    input_text = train_list['input'][i]
    reference_summary = train_list['output'][i]

    generated_summary = generate_discharge_summary(input_text)

    generated_summaries.append(generated_summary)
    reference_summaries.append(reference_summary)

# Compute BLEU scores
bleu = sacrebleu.corpus_bleu(generated_summaries, [reference_summaries])

The patient is a year old male who complains of low back pain and previous laminectomy L2-L5.
A patient presents with a history of multiple hospital admissions for intractable headache, nausea, vomiting, vertigo, weakness, and previous diagnosis of IIH based on elevated LP opening pressure.
Key words: supracervical hysterectomy, pelvic pain, vaginal bleeding, surgery, emergency department.
Key words: ERCP, cholangiocarcinoma, abdominal pain.
The patient has BPD and chronic LBP and has been taking percocets and taken 30 tablets over the past few weeks, with no head trauma or LOC.
Key words: aortic valve repair, transfemoral aortic valve repair, open heart surgery, New York Heart Association.
The patient presented to the emergency department (MICU) with a history of exacerbations of COPD and a recent bout of pneumonia.
Key words: fluid overload, coma, anoxic brain injury, pancreatitis, PEG, trach, ventilator, discharge.
Key words: lymphoma, chemotherapy, central nervous system (CNS) lymp

In [ ]:
print("\n👉 BLEU Score:", bleu.score)

# Compare some examples
for i in range(5):
    print(f"\nExample {i + 1}:")
    print("Input Text:", train_list['input'][i])
    print("Generated Summary:", generated_summaries[i])
    print("Reference Summary:", reference_summaries[i])


👉 BLEU Score: 0.04845332067011967

Example 1:
Input Text: Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: M Service: MEDICINE Allergies: naproxen / ibuprofen Attending: ___. Chief Complaint: Back pain Major Surgical or Invasive Procedure: None this hospitalization History of Present Illness: HISTORY OF PRESENT ILLNESS: This patient is a ___ year old male who complains of low back pain w/ herniated disc (L 3-L4) and previous laminectomy L2-L5 last year @ ___- presents w/ ___ pain- difficulty walking. Patient states that he has been having left-sided back pain radiating down to his left hip. No inc stool/urine. +chronic constipation (not assoc with onset of back pain). Took 25mg Oxycodone last night and fell OOB after this- No head trauma- Usually takes 20mg Oxycodone for pain exacerbations. No fevers or chills. No chest pain or shortness of breath. In the ED, initial vitals were: Temp: 98.8 HR: 101 BP: 132/74 Resp: 16 O(2)Sat: 95 Normal On the flo

In [ ]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00


In [ ]:
import evaluate
# Load ROUGE metric
rouge = evaluate.load('rouge')

# Calculate ROUGE scores
results = rouge.compute(predictions=generated_summaries, references=reference_summaries)

print("\n👉  ROUGE Scores:", results)

# Compare some examples
for i in range(5):
    print(f"\nExample {i + 1}:")
    print("Input Text:", train_list['input'][i])
    print("Generated Summary:", generated_summaries[i])
    print("Reference Summary:", reference_summaries[i])


👉  ROUGE Scores: {'rouge1': 0.06281995422258685, 'rouge2': 0.008591768267735864, 'rougeL': 0.040757630829215005, 'rougeLsum': 0.04048791654844759}

Example 1:
Input Text: Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: M Service: MEDICINE Allergies: naproxen / ibuprofen Attending: ___. Chief Complaint: Back pain Major Surgical or Invasive Procedure: None this hospitalization History of Present Illness: HISTORY OF PRESENT ILLNESS: This patient is a ___ year old male who complains of low back pain w/ herniated disc (L 3-L4) and previous laminectomy L2-L5 last year @ ___- presents w/ ___ pain- difficulty walking. Patient states that he has been having left-sided back pain radiating down to his left hip. No inc stool/urine. +chronic constipation (not assoc with onset of back pain). Took 25mg Oxycodone last night and fell OOB after this- No head trauma- Usually takes 20mg Oxycodone for pain exacerbations. No fevers or chills. No chest pain or shortnes

##Training

In [ ]:
from transformers import AutoTokenizer

def tokenize_function(examples):
    inputs = examples["input"]
    targets = examples["output"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize targets with the same parameters
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train_dataset = train.map(tokenize_function, batched = True)
tokenized_valid_dataset = valid.map(tokenize_function, batched = True)

tokenized_train_dataset = tokenized_train_dataset.remove_columns(['input', 'output'])
tokenized_valid_dataset = tokenized_valid_dataset.remove_columns(['input', 'output'])

tokenized_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_valid_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/7940 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/992 [00:00<?, ? examples/s]

In [ ]:
len(tokenized_train_dataset['attention_mask'])

7940

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, DataCollatorForSeq2Seq, Trainer, TrainingArguments
# Load base model for causal LM and apply quantization config
device = "cuda:0" if torch.cuda.is_available() else "cpu"
#device = "cpu"
BATCH_SIZE = 5
EPOCH = 2

# Configure LoraConfig for model training
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# Define data collator
collate_fn = DataCollatorForSeq2Seq(tokenizer, return_tensors="pt", padding=True, model = model)

STEPS_PER_EPOCH = len(tokenized_train_dataset) // BATCH_SIZE

In [ ]:
from transformers import Trainer, TrainingArguments

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # Set batch size to 1
    per_device_eval_batch_size=1,  # Set batch size to 1
    gradient_accumulation_steps=16,  # Simulate a larger batch size
    eval_accumulation_steps=2,
    do_train=True,
    do_eval=True,
    logging_steps=500,
    save_steps=500,
    eval_steps=500,
    num_train_epochs=2,
    eval_strategy="steps",
    save_total_limit=3,
    fp16=True,  # Use mixed precision training
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    tokenizer=tokenizer,
    data_collator=collate_fn,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
500,8.643500,7.577553


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=992, training_loss=8.24517551545174, metrics={'train_runtime': 4740.4376, 'train_samples_per_second': 3.35, 'train_steps_per_second': 0.209, 'total_flos': 2.3083712350519296e+16, 'train_loss': 8.24517551545174, 'epoch': 1.998992443324937})

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

Evaluation Results: {'eval_loss': 7.461764335632324, 'eval_runtime': 87.9298, 'eval_samples_per_second': 11.282, 'eval_steps_per_second': 11.282, 'epoch': 1.998992443324937}


In [ ]:
import os

# Specify the directory where the model should be saved
model_dir = "./results"

# List the contents of the directory
if os.path.exists(model_dir):
    print("Model directory contents:", os.listdir(model_dir))
else:
    print("Model directory does not exist.")

Model directory contents: ['checkpoint-992', 'runs', 'checkpoint-500']


In [ ]:
test[0]["input"]

'Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: F Service: MEDICINE Allergies: No Known Allergies / Adverse Drug Reactions Attending: ___. Chief Complaint: Abdominal Pain Major Surgical or Invasive Procedure: ___ ERCP History of Present Illness: This patient is a ___ year old female who complains of RUQ abdominal pain. ___ hx HTN, hypothyroidism, recent ___ placement presenting with 3 days of RUQ pain. Pain is worse with eating, radiates to back. Massage helps. Reports constant ___ pain and she feels "gassy." Reports history of intermittent RUQ pain for years. Previously diagnosed with cholelithiasis. Last BM was this AM. Denies hx abdominal surgeries. Seen with ipad ___ interpreter ___ ___. Denies fevers, chills, cp, sob, nausea, vomiting, diarrhea, melena, BRBPR. Past Medical History: PAST MEDICAL HISTORY: -gallstones -hypothyroidism -peripheral vascular disease PAST SURGICAL HISTORY: -cardiac stents Social History: ___ Family History: Family H

In [ ]:
test_input = test[0]["input"]
inputs = tokenizer(test_input, return_tensors='pt', truncation=True, padding='longest').to(device)
summary_ids = model.generate(inputs['input_ids'], max_length=200, num_beams=4, length_penalty=2.0, early_stopping=True)
generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(f"Generated Summary: {generated_summary}")

Generated Summary: The patient is a year old female who complains of RUQ abdominal pain for years.


In [ ]:
test[0]["output"]

'Dear Ms. ___, You were admitted to ___ with abdominal pain and found to have a narrowing in your bile duct that drains from the gallbladder. You underwent a procedure to have a ___ placed in the duct and allow the bile to flow and samples were taken. General Surgery: Please call your doctor or nurse practitioner or return to the Emergency Department for any of the following: *You experience new chest pain, pressure, squeezing or tightness. *New or worsening cough, shortness of breath, or wheeze. *If you are vomiting and cannot keep down fluids or your medications. *You are getting dehydrated due to continued vomiting, diarrhea, or other reasons. Signs of dehydration include dry mouth, rapid heartbeat, or feeling dizzy or faint when standing. *You see blood or dark/black material when you vomit or have a bowel movement. *You experience burning when you urinate, have blood in your urine, or experience a discharge. *Your pain in not improving within ___ hours or is not gone within 24 hou